<a href="https://colab.research.google.com/github/robinjameslee/Calculating-the-Implied-Volatility-for-European-Option-in-Python/blob/main/Calculating_the_Implied_Volatility_for_European_Option_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates how to calculate the implied volatility of a European Option


In [35]:
# import numpy as np
from numpy import sqrt, log, exp, pi
from scipy.stats import norm

N = norm.cdf

# 1. First we create a function to calculate our european option with black-scholes formula
# S = spot
# K = strike
# t = time to maturity
# r = risk-free rate
# sigma = annualized volatility
# option_type = Call or Put

def bs_price(S, K, t, r, sigma, option_type):
    d1 = (log(S / K) + (r + sigma ** 2 / 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)

    # black-scholes formula
    if option_type == 'Call':
        return N(d1) * S - N(d2) * K * exp(-r * t)
    elif option_type == 'Put':
        return N(-d2) * K * exp(-r * t) - N(-d1) * S

In [36]:
# For our example, let's consider the following:
S = 100
K = 110
t = 1 # one year till maturity
r = 0.03 # 3% interest rates
sigma = 0.35 # annualized volatility
option_type = 'Call'

our_bs_price = bs_price(S, K, t, r, sigma, option_type)
print(f"our option's price is: {our_bs_price}")

our option's price is: 6.611207651952377


In [37]:
# 2. Let's try to calculate the implied volatility from the option price using brue force
brute_tolerance = 0.0005 # We'll stop calculate if difference between the market price and our calculated price is less than this
brute_iter_step = 0.0005 # We move each iteration by this size
brute_iterations = 10000 # iterate it 10000 times

def brute_force(S, K, t, r, option_type, mkt_price):
    sigma = 0.5 # initial value
    for i in range(brute_iterations): 
        iter_price = bs_price(S, K, t, r, sigma, option_type)
        diff = mkt_price - iter_price
        if abs(diff) < brute_tolerance:
            return sigma
        elif diff > brute_tolerance:
            sigma += brute_iter_step
        else:
            sigma -= brute_iter_step
    return sigma

brute_iv = brute_force(S, K, t, r, option_type, our_bs_price)
print(f"our brute force implied volatility is: {brute_iv}")

our brute force implied volatility is: 0.34999999999999987


In [38]:
# We can also use the Newton Raphson Algorithm to calculate the implied volatility
# This method uses vega as part of its calculation
newton_tolerance = 0.0005 # We'll stop calculate if difference between the market price and our calculated price is less than this
newton_iterations = 10000 # iterate it 10000 times

def vega(S, K, t, r, sigma):
    d1 = (log(S / K) + (r + sigma ** 2 / 2) * t) / (sigma * sqrt(t))
    vega = S  * sqrt(t) * norm.pdf(d1)
    return vega

def newton_iv(S, K, t, r, option_type, mkt_price):
    sigma = 0.5 # initial value
    for i in range(newton_iterations):
        iter_price = bs_price(S, K, t, r, sigma, option_type)
        diff = mkt_price - iter_price
        local_vega = vega(S, K, t, r, sigma)
        if abs(diff) < newton_tolerance:
            return sigma
        sigma += diff/local_vega
    return sigma

newton_iv = newton_iv(S, K, t, r, option_type, our_bs_price)
print(f"our newton implied volatility is: {newton_iv}")

our newton implied volatility is: 0.35000013279439707
